# Исследование надежности заемщиков


## 1 Обзор данных

In [1]:
# импорт библиотеки и открытие файла
import pandas as pd

data = pd.read_csv('E:\My documents\Learning\Data_Analyst\Projects\pr_2_credit_department/data.csv')

In [2]:
# вывод первых 10 строк датафрейма
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# вывод основной информации о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2 Предобработка данных

### Удаление пропусков

In [4]:
# подсчет количества пропусков в каждом из столбцов
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
# заполнение пропусков в столбце с уровнем дохода его медианным значением по типу занятости
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
# замена отрицательных значений количества дней трудового стажа на положительные
data['days_employed'] = data['days_employed'].abs()

In [7]:
# расчет медианного значения трудового стажа по типу занятости
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их без изменения.

In [8]:
# вывод уникальных значений количества детей
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [9]:
# удаление аномальных значений количества детей (-1, 20)
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
# проверка удаления аномальных значений
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [11]:
# заполнение пропусков в столбце со значением трудового стажа его медианным значением по типу занятости
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
# проверка устранения пропусков
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
# замена вещественного типа данных на целочисленный
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
# приведение значений столбца с уровнем образования к нижнему регистру
data['education'] = data['education'].str.lower()

In [15]:
# вывод количества строк-дубликатов
data.duplicated().sum()

71

In [16]:
# удаление дубликатов
data = data.drop_duplicates()

### Категоризация данных

In [17]:
# создание функции для распределения кредитополучателей по категориям уровня дохода
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
# создание нового столбца с применением функции к столбцу с уровнем дохода
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
# вывод уникальных целей взятия кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
# создание функции для распределения целей взятия кредита по категориям
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
# создание нового столбца с применением функции к столбцу с целью взятия кредита
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## 3 Анализ зависимостей

In [22]:
# расчет процентного соотношения числа людей, имевших задолженности, к общему числу людей с группировкой по количеству детей
(data.groupby('children')['debt'].sum() / data.groupby('children')['debt'].count() * 100).sort_values(ascending=False)

children
4    9.756098
2    9.454191
1    9.234609
3    8.181818
0    7.543822
5    0.000000
Name: debt, dtype: float64

**Вывод:** Исходя из полученных значений, можно сделать вывод о том, что количество детей оказывает несильное влияние на возврат кредита в срок, так как разброс между группами, имеющими 1, 2, 3 и 4 детей, относительно небольшой. Кроме того, в группе людей, имеющих 5 детей, вообще никто не имел задолженности по возврату кредитов.

In [23]:
# расчет процентного соотношения числа людей, имевших задолженности, к общему числу людей с группировкой по семейному положению
(data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count() * 100).sort_values(ascending=False)

family_status
Не женат / не замужем    9.763948
гражданский брак         9.313014
женат / замужем          7.560558
в разводе                7.064760
вдовец / вдова           6.624606
Name: debt, dtype: float64

**Вывод:** На основании произведенного расчета можно сделать вывод о том, что семейное положение не имеет большой взаимосвязи с возратом кредита в срок, так как процент количества людей, имевших задолженности, в группах: "Не женат / не замужем" (9,8%) и "гражданский брак" (9,3%), а также в "в разводе" (7,1%) и "женат / замужем" (7,6%) - не сильно отличаются.

In [24]:
# расчет процентного соотношения числа людей, имевших задолженности, к общему числу людей с группировкой по уровню дохода
(data.groupby('total_income_category')['debt'].sum() / data.groupby('total_income_category')['debt'].count() * 100).sort_values(ascending=False)

total_income_category
E    9.090909
C    8.498210
A    8.000000
B    7.060231
D    6.017192
Name: debt, dtype: float64

In [25]:
# подсчет доли каждой категории в общем количестве людей, имевших задолженности
(data.groupby('total_income_category')['debt'].sum() / data['debt'].sum() * 100).sort_values(ascending=False)

total_income_category
C    78.117783
B    20.438799
D     1.212471
A     0.115473
E     0.115473
Name: debt, dtype: float64

**Вывод:** Ожидаемо, процент людей, имевших задолженности, выше всего в категории Е (с уровнем дохода до 30.000), однако, наименьшее значение данный показатель имеет не в категории А (с уровнем дохода от 1.000.000 и выше), а в категории D (от 30.001 до 50.000). Таким образом, нельзя говорить о наличии высокой взаимосвязи между уровнем дохода и возвратом кредита в срок.

Кроме того, если отдельно проанализировать процент в общем числе людей, имевших задолженности, по категориям, то бОльшую долю (а именно 78,1%) составляет группа С (с уровнем дохода 50.001 до 200.000).

In [26]:
# расчет процентного соотношения числа людей, имевших задолженности, к общему числу людей с группировкой по цели кредита
(data.groupby('purpose_category')['debt'].sum() / data.groupby('purpose_category')['debt'].count() * 100).sort_values(ascending=False)

purpose_category
операции с автомобилем      9.347978
получение образования       9.252758
проведение свадьбы          7.911803
операции с недвижимостью    7.255139
Name: debt, dtype: float64

**Вывод:** Процент невозврата кредита в срок выше в категориях "операции с автомобилем" и "получение образования", но имеет небольшое отклонение от категорий "проведение свадьбы" и "операции с недвижимостью".

### Общий вывод

По результатам проведенного анализа нельзя говорить о том, что такие показатели, как количество детей, семейное положение, уровень дохода и цель кредита, оказывают сильное влияние на возврат кредита в срок. Для более подробного изучения взаимосвязей с этим показателем следует изучить такие критерии, как уровень образования, возраст и наличие/отсутствие судимости.